# Metropolis-Hastings chaotic amplitude control

Timothée Guillaume Leleu, Sam Reifenstein

Published in the proceedings of ICLR2025

In [1]:
import tune_wishart_wrap
import numpy as np
#from tqdm import tqdm
from tqdm.notebook import tqdm
import time
import itertools
import lib

import os

biasl = [0.0,12.0]
Nl = [60,100,120,140]
T = [50,100,300,500,1000,2000,3000,4000]
#T = [50,100,300]


pt_device = 'cuda'
#pt_device = 'cpu'

alphatxt = '0.80'

fMHv = {}
fMHv['CACm'] = 0.0
fMHv['AIM'] = 0.0
fMHv['CAC'] = 0.0
fMHv['MHCACm'] = 0.1
fMHv['MHCAC'] = 0.1

####################################################
#SOLVER

solvertypel = ['MHCACm','CACm','AIM','CAC']

debug = 0

for bias in biasl:

    data = {}
    if bias==0:
        data['datatype'] = 'unbias'
    else:
        data['datatype'] = 'bias'

    if data['datatype'] == 'unbias':
        data['D_WPE'] = 1
        data['R_WPE'] = -1
        data['bias'] = 0.0
        
    elif data['datatype'] == 'bias':
        data['D1_WPE'] = 1
        data['R1_WPE'] = 3
        data['D2_WPE'] = 1
        data['R2_WPE'] = -1
        data['bias'] = bias
        data['D_WPE'] = data['D1_WPE'] + data['D2_WPE']
        bias = data['bias']

    ####################################################

    total = len(Nl) * len(T)

    afolder_name = []

    for solvertype in solvertypel:

        print(f'Computing {solvertype}')

        fMH = fMHv[solvertype]

        folder_name = f"{solvertype}_{bias}_{fMH}"
        
        # Check if the folder already exists
        if not os.path.exists(folder_name):
            # Create the folder if it doesn't exist
            os.makedirs(folder_name)
            print(f"Folder created: {folder_name}")
        else:
            print(f"Folder already exists: {folder_name}")
            
        # Iterate over all combinations
        for combination in tqdm(itertools.product(Nl, np.flip(T)), desc="Iterating combinations"):
        
            if solvertype=='MHCACm':
                PARAM_NAMES = ["beta","kappa","lamb","xi","gamma","a"]
                x = np.log([0.1,0.1,1.0,0.1,1.0,1.0])
                hyperparams = {'T': combination[1],'doa':1,'dosampling':0,'fMH':fMH}
                fMH = hyperparams['fMH']
                
            if solvertype=='MHCAC':
                PARAM_NAMES = ["beta","kappa","lamb","xi","a"]
                x = np.log([0.1,0.1,0.1,0.1,1.0])
                hyperparams = {'T': combination[1],'doa':0,'dosampling':0,'fMH':1.0}
                fMH = hyperparams['fMH']
                
            if solvertype=='CACm':
                PARAM_NAMES = ["beta","lamb","xi","gamma","a"]
                x = np.log([0.1,1.0,0.1,1.0,1.0])
                hyperparams = {'T': combination[1]}
            
            if solvertype=='CAC':
                PARAM_NAMES = ["beta","lamb","xi","a"]
                x = np.log([0.1,1.0,0.1,1.0])
                hyperparams = {'T': combination[1]}
            
            if solvertype=='AIM':
                PARAM_NAMES = ["beta","lamb","gamma"]
                x = np.log([0.1,1.0,1.0])
                hyperparams = {'T': combination[1]}
                
            #generate problem instance
            instance = {'alphatxt':alphatxt, 'N': combination[0]}
            flags = {'savetraj':1, 'pt_device': pt_device,'solvertype':solvertype}
            tunerparams = {'nsamp_max': 100000, 'R': 200}
            
            tune_wishart_wrap.tune_wishart(folder_name,instance,hyperparams,PARAM_NAMES,x,flags,tunerparams,data)

        afolder_name.append(folder_name)


TabError: inconsistent use of tabs and spaces in indentation (DASTuneADAM.py, line 169)

# Figure 2 : 

The time to solution of (unbiased) Wishart planted instances and biased degenerate planted instance

In [ ]:
    
import matplotlib.pyplot as plt
plt.style.use('plot_style.txt')

# Construct the folder name
figfolder = "fig_tune_wishart"

# Check if the folder already exists
if not os.path.exists(figfolder):
    # Create the folder if it doesn't exist
    os.makedirs(figfolder)
    print(f"Folder created: {figfolder}")
else:
    print(f"Folder already exists: {figfolder}")

    
plt.figure(figsize=(12, 4))

colors = ['r','g','b','m','y']
markers = ['d','s','o','x','<']
lines = ['-','--',':','-.','--']

to_plot = ['unbias','bias']

####################################################

for ib, biastxt in enumerate(to_plot):  

    data = {}
    data['datatype'] = biastxt
    
    bias = 0.0
    
    if data['datatype'] == 'unbias':
        data['D_WPE'] = 1
        data['R_WPE'] = -1
        data['bias'] = 0.0
        
    elif data['datatype'] == 'bias':
        data['D1_WPE'] = 1
        data['R1_WPE'] = 3
        data['D2_WPE'] = 1
        data['R2_WPE'] = -1
        data['bias'] = 12.0
        data['D_WPE'] = data['D1_WPE'] + data['D2_WPE']
        bias = data['bias']
    
    ####################################################
    
    res_sol = []
    
    for solvertype in solvertypel:
    
        res = []
        
        fMH = fMHv[solvertype]
    
        for N in Nl:
    
            p0mat = np.zeros(len(T))
            TTSmat = np.zeros(len(T))
            TTSmatv = np.zeros(len(T))

            for iT, cT in enumerate(T):
    
                folder_name_ = f"{solvertype}_{bias}_{fMH}"
                file_name = f"wishart_{N}_{alphatxt}_{bias}_{cT}_{fMH}.txt"
                
                #print(file_name)
    
                p0, opt_params, pvec = lib.read_file(folder_name_, file_name)
    
                if solvertype=='AIM' and N==100:
                    print('AIM',bias,p0)
    
                #bootstrap
                if len(opt_params)>0:
                    p0mat[iT] = p0
                    TTS = np.mean(np.log(1-0.99)/np.log(1-np.array(pvec))*cT)
                    TTSmat[iT] = TTS
    
                    TTSv = np.std(np.log(1-0.99)/np.log(1-np.array(pvec))*cT)
                    TTSmatv[iT] = TTSv*1.96/np.sqrt(len(pvec))
    
                    print("T:", T, "p0:", p0, "TTS:", TTS)
    
            res.append((p0mat,TTSmat,TTSmatv))
            
        res_sol.append(res)
        
    ####################################################

    
    solvertypel2 = ['MHCACm','CACm']

    for res, solvertype, color, line in zip(res_sol,solvertypel2,colors,lines):
    
        for cres, N, marker in zip(res,Nl,markers):
    
            p0mat,TTSmat,TTSmatv = cres
            
            TTSmat[TTSmat==0] = np.nan
            TTSmatv[TTSmat==0] = np.nan

    ax = plt.subplot(1,2,ib+1)

    plt.rcParams['figure.figsize'] = [6, 4] 
    
    for res, solvertype, color, line, marker in zip(res_sol,solvertypel,colors,lines, markers):
    
        TTS_CAC = []
        TTS_CACv = []
        
        for cres, N in zip(res,Nl):
        
            p0mat,TTSmat,TTSmatv = cres
            
            TTSmat[TTSmat<=0] = np.nan
            TTSmatv[TTSmat<=0] = np.nan
            
            if np.sum(np.isnan(TTSmat))<len(T):
                imin = np.nanargmin(TTSmat)
                TTS_CAC.append(TTSmat[imin])
                TTS_CACv.append(TTSmatv[imin])
            else:
                TTS_CAC.append(np.nan)
                TTS_CACv.append(np.nan)
        
        TTS_CAC = np.array(TTS_CAC)
        TTS_CACv = np.array(TTS_CACv)
        
        plt.plot(Nl,TTS_CAC,linestyle=line,marker=marker,color=color,label='%s' % solvertype)
        
        plt.fill_between(Nl,TTS_CAC-TTS_CACv,TTS_CAC+TTS_CACv,color=color, interpolate=True, alpha=0.3)
    
    plt.yscale('symlog')
    plt.xlabel(r'N')
    plt.ylabel('TTS (any ground-state)')
    plt.legend(ncols=2)
    
    plt.xlim(np.min(Nl),np.max(Nl))
    
    plt.ylim(10**2,10**7)
    
    plt.gca().spines["top"].set_visible(True)
    plt.gca().spines["right"].set_visible(True)
    
    plt.grid(True)
    
    if ib==0:
        plt.title('a (unbiased Wishart)')
    if ib==1:
        plt.title('b (biased Wishart, b=%d)' % bias)
            
    
